In [1]:
import os
import sys
sys.path.append(os.path.abspath('../python'))

import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from models.non_constant_diffusivity import NonConstantDiffusivityNeuralNetwork
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [ ]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/model_paper')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

In [ ]:
# Load dataset
dataset = load_data(DATA_PATH)

In [4]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

In [6]:
# Tratamiento de los datos para dividirlos en train y test
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = TensOps(torch.tensor(dataset['y_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_train = TensOps(torch.tensor(dataset['k_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_train = TensOps(torch.tensor(dataset['f_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_np = X_train
y_np = y_train.values
K_np = K_train.values
f_np = f_train.values

X_train_np, X_test_np, y_train_np, y_test_np, K_train_np, K_test_np, f_train_np, f_test_np = train_test_split(X_np, y_np, K_np, f_np, test_size=0.2, random_state=42)

X_train = X_train_np.to(device)
X_test = X_test_np.to(device)

y_train = TensOps(y_train_np.to(device), space_dimension=y_train.space_dim, contravariance=0, covariance=0)
y_test = TensOps(y_test_np.to(device), space_dimension=y_train.space_dim, contravariance=0, covariance=0)

K_train = TensOps(K_train_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)
K_test = TensOps(K_test_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)

f_train = TensOps(f_train_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)
f_test = TensOps(f_test_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)

In [7]:
# Arquitectura del modelo
input_shape = X_train[0].shape  # [1, 10, 8]
hidden1_dim = 150
hidden2_dim = 150
output_shape = y_train.values[0].shape  # [1, 10, 10]

In [ ]:
# Se carga el modelo y el optimizador
model = NonConstantDiffusivityNeuralNetwork(input_shape, hidden1_dim, hidden2_dim, output_shape).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 10001

batch_size = 64
n_checkpoints = 100

train_loop(model, optimizer, n_checkpoints,
           X_train, y_train, X_test, y_test, f_train, f_test,
           D=D, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=device,
        )

In [ ]:
# Parametros de entrenamiento
start_epoch = 10000
n_epochs = 110001

batch_size = 64
n_checkpoints = 100

second_lr = 3e-5

train_loop(model, optimizer, n_checkpoints,
           X_train, y_train, X_test, y_test, f_train, f_test,
           D=D, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=device,
        )